# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

    tracking server: yes, local server
    backend store: sqlite database
    artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [2]:
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Tracking URI: http://127.0.0.1:5000


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/31 14:11:21 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-2' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/1/aac440d418e940b7a818c8e48e0fd6c8/artifacts'


/Users/abhinav/miniforge3/envs/ml/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Interacting with Model Registry

In [4]:
from mlflow.tracking import MlflowClient
client = MlflowClient('http://127.0.0.1:5000')

In [5]:
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Tracking URI: http://127.0.0.1:5000


In [6]:
client.search_registered_models()

[]

# Register a Model

In [20]:
result = mlflow.search_runs(experiment_names=["my-experiment-2"])
run_id = result.loc[0, "run_id"]
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)


Successfully registered model 'iris-classifier'.
2023/05/31 20:28:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685545130287, current_stage='None', description='', last_updated_timestamp=1685545130287, name='iris-classifier', run_id='aac440d418e940b7a818c8e48e0fd6c8', run_link='', source='mlflow-artifacts:/1/aac440d418e940b7a818c8e48e0fd6c8/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>